In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
import pandas as pd

In [ ]:
weather = pd.read_csv('weather.csv', parse_dates=[0], dayfirst=True)
weather = weather.set_index('Date')
weather.head(5)

In [ ]:
climate = pd.read_csv('Climate_2012to2022.csv', parse_dates=[0], dayfirst=True)
climate.rename(columns = {'dateV':'Date'}, inplace = True)
climate = climate.set_index('Date')
climate.head(5)

In [ ]:
weather_and_climate = pd.merge(weather, climate, left_index=True, right_index=True, how='outer')
weather_and_climate.tail(5)
weather_and_climate = weather_and_climate.drop(weather_and_climate.columns[[0,1,2,3,4,5,20]], axis=1)
weather_and_climate = weather_and_climate[3:]
weather_and_climate = weather_and_climate.resample('W-SAT', closed='right').mean()
weather_and_climate.info()

In [ ]:
from epiweeks import Week, Year
from datetime import date
def create_epiweek(date):
    return Week.fromdate(date)
weather_and_climate['epiweekEndDate'] = weather_and_climate.index.date
weather_and_climate['Epiweek'] = weather_and_climate['epiweekEndDate'].apply(create_epiweek)

In [ ]:
weather_and_climate = weather_and_climate.set_index('Epiweek')
weather_and_climate.to_csv("weather_and_climate.csv")

In [ ]:
weather_and_climate.info()
start_week = Week(2009,1)
end_week = Week(2009,52)
MaxT = weather_and_climate['MaxT'].dropna().loc[start_week:end_week]
#ts = str(weather_climate_ED['Epiweek'].dropna().loc[start_date:endl_date])
plt.plot(MaxT)

In [ ]:
ED = pd.read_csv('EDbySBODcat.csv', parse_dates=[0], dayfirst=True)
ED = ED.dropna()
ED.rename(columns = {'date_adm':'Date'}, inplace = True)
ED.head(5)

In [ ]:
EDs = dict(tuple(ED.groupby('SBoDcate')))
ED_list = list(EDs.keys())
for category in ED_list:
    EDs[category] = EDs[category].set_index('Date')
    EDs[category].rename(columns = {'count': 'SBoD-'+category}, inplace=True)
    EDs[category].drop(columns = ['SBoDcate'], inplace=True)
merged_ED = pd.concat([EDs[x] for x in ED_list], axis=1)
merged_ED.to_csv('merged_ED.csv')

In [ ]:
weather_climate_ED = pd.merge(weather_and_climate, merged_ED, left_index=True, right_index=True, how='outer')
weather_climate_ED.tail(5)

In [ ]:
weather_climate_ED["Year"] = weather_climate_ED.index.year
weather_climate_ED["Month"] = weather_climate_ED.index.month
weather_climate_ED["Day"] = weather_climate_ED.index.day
weather_climate_ED["DOW"] = weather_climate_ED.index.dayofweek+1
def dow_adjust(dow):
    if dow == 7:
        return 0
    else:
        return dow
weather_climate_ED["DOW"] = weather_climate_ED["DOW"].apply(dow_adjust)
dow_map = {0:'Sunday', 1:'Monday', 2:'Tuesday', 3:'Wednesday', 4:'Thursday', 5:'Friday', 6:'Saturday'}
weather_climate_ED['DOW.Name'] = weather_climate_ED['DOW'].map(dow_map)

In [ ]:
from epiweeks import Week, Year
from datetime import date
def create_epiweek(date):
    return Week.fromdate(date)
weather_climate_ED['Date'] = weather_climate_ED.index.date
weather_climate_ED['Epiweek'] = weather_climate_ED['Date'].apply(create_epiweek)
weather_climate_ED.to_csv('weather_climate_ED.csv')
weather_climate_ED.info()

In [ ]:
epiweek_list = weather_climate_ED['Epiweek'].unique()
epiweek_data = pd.DataFrame(index = epiweek_list)
epiweek_data.info()
epiweek_data['MaxT'] = 

In [ ]:
exp_col = ['SBoD-Chronic respiratory disease', 'SBoD-Respiratory Infection']
start_date = '2017-01-01'
end_date = '2017-12-31'
chronic_resp = weather_climate_ED[exp_col[0]].dropna().loc[start_date:end_date]
ts = str(weather_climate_ED['Epiweek'].dropna().loc[start_date:end_date])
plt.plot(ts, chronic_resp)
chronic_resp.info()

In [ ]:
mu = np.mean(chronic_resp)
sigma = np.std(chronic_resp)
chronic_resp = (chronic_resp - mu) / sigma

In [ ]:
def perform_adf_test(series):
    result = adfuller(series)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])

In [ ]:
perform_adf_test(chronic_resp)

In [ ]:
plot_pacf(chronic_resp, lags=14, method='ols')
plt.show()

In [ ]:
plot_acf(chronic_resp)
plt.show()

In [ ]:
model = ARIMA(chronic_resp, order=(1,0,4)).fit()
print(model.params)

In [ ]:
prediction_info = model.forecast(3)
print(prediction_info)

In [ ]:
plt.plot(chronic_resp.tail(30))
plt.plot(prediction_info)